In [ ]:
import ee
import geemap


In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='ee-imhayatkhan')

In [ ]:
# This is shortcut for the above two lines
geemap.ee.Initialize()

In [5]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

In [19]:
m = geemap.Map(center = [53.3811, 1.4701], zoom = 8) # UK
m

Map(center=[53.3811, 1.4701], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [22]:
m = geemap.Map(data_ctrl=False,toolbar_ctrl=False, search_ctrl=False, draw_ctrl=False, measure_ctrl=False, layer_ctrl=False, geocoder_ctrl=False,fullscreen_ctrl=False, home_ctrl=False, add_data_ctrl=False, print_ctrl=False, time_slider_ctrl=False, inspector_ctrl=False, add_marker_ctrl=False, add_polygon_ctrl=False, add_rectangle_ctrl=False, add_circle_ctrl=False, add_ee_layer_ctrl=False, add_text_ctrl=False, add_image_marker_ctrl=False, add_geojson_file_ctrl=False) # UK
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# Adding Basemaps

In [24]:
m = geemap.Map(basemap = 'Esri.WorldImagery')
m.add_basemap('Esri.WorldTopoMap')
m.add_basemap('OpenTopoMap')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [26]:
# Listing Available Basemaps
basemap = list(geemap.basemaps.keys())
print(f'Total basemaps Available: {len(basemap)}')

Total basemaps Available: 150


In [27]:
basemap[0:5] # First 5 Basemaps

['OpenStreetMap',
 'Esri.WorldStreetMap',
 'Esri.WorldImagery',
 'Esri.WorldTopoMap',
 'FWS NWI Wetlands']

# Interactive basemap Select

In [28]:
m = geemap.Map()
m.add('basemap_selector')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Layer Manager

In [ ]:
m = geemap.Map( center = (40,-100), zoom = 4)
dem = ee.Image('USGS/SRTMGL1_003') # the String is the unique ID of the image in the Earth Engine data catalog
states = ee.FeatureCollection('TIGER/2018/States') # the string is the unique ID of the featue collection (shape file ) in the google earth engine data catalog 

vis_params = {
    'min' : 0,
    'max' : 4000,
    'palette' : ['blue', 'green', 'yellow', 'orange', 'red']
}
#m.addLayer(dem, vis_params, 'SRTM DEM')
m.add_layer(dem, vis_params, 'SRTM DEM') # m.addLayer is the same as m.add_layer
m.addLayer(states, {}, 'States')    
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

# Inspector Tool

In [29]:
m = geemap.Map(center = (40,-100), zoom = 4)
dem = ee.Image('USGS/SRTMGL1_003') # the String is the unique ID of the image in the Earth Engine data catalog
states = ee.FeatureCollection('TIGER/2018/States') # the string is the unique ID of the featue collection (shape file ) in the google earth engine data catalog 
lansat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003') # the String is the unique ID of the image in the Earth Engine data catalog
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
    }
m.addLayer(dem,vis_params,'SRTM DEM')
m.addLayer(states, {}, 'States')
m.addLayer(lansat7, 
           {
                'bands': ['B4', 'B3', 'B2'],
                'min': 20,
                'max': 200,
                'gamma': 2.0
           },   'Landsat 7')
m.add('inspector')
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

# Layer Editor

In [31]:
m = geemap.Map(center = (40,-100), zoom = 4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min' : 0,
    'max' : 4000,
    'palette' : ['blue', 'green', 'yellow', 'orange', 'red']
}
m.addLayer(dem, vis_params, 'SRTM DEM')

m


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

<IPython.core.display.Javascript object>

In [ ]:
# The code has been copied to the clipboard. 
# Press Ctrl+V to in a code cell to paste it.
dataset = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

#Map.setCenter(12.876, 42.682, 5)



m.addLayer(dataset, {}, 'Second Level Administrative Units')


Export started with task ID: R34GQS6AI5XCCJU7LP643QM5
